In [1]:
"""
Made in Python 2.7, should also be compatible with Python 3, but not guaranteed!
Written by Dieter Joubert
"""

'\nMade in Python 2.7, should also be compatible with Python 3, but not guaranteed!\nWritten by Dieter Joubert\n'

In [2]:
#we'll need urllib2 to open and get a webpage's data, and BeautifulSoup to scrape it, so we import those
import urllib2
from bs4 import BeautifulSoup

#we'll use pandas to hold the data we scrape and do some further clean up
import pandas as pd

#the url of the page we want to scrape
url = "http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2005"

#let urllib2 open the website and hold a reference to it
page = urllib2.urlopen(url)

#and run BeautifulSoup on it, we are now ready to begin scraping! (you might get a warning, but that's not a problem)
soup = BeautifulSoup(page)

/Users/dieterjoubert/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Now let's take a look at the page we want to scrape: http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2005

We want to scrape the table on that page into a CSV table. But scraping the table isn't always so obvious and easy. So let's go ahead and look at the actual HTML of the page. You can do this with most web browsers by, while being on the page itself, right-clicking and selecting "View Page Source." I recommend copy-pasting this html into http://www.dirtymarkup.com/ and hitting "Clean" (this re-organizes the html to standardize indenting, etc. to make it easier to read). 

There is a clean-up copy in this folder, go ahead and open it with a text-editor to see the html.
    
After some searching around we see the following HTML code:

In [3]:
""" 
<table border="0" cellpadding="2" cellspacing="1"
                        class="ScheduleTable" width="100%">
                            <tr>
                                <th align="center" class="subhdr" scope="col">
                                Date</th>
                                <th align="center" class="subhdr" scope="col">
                                Opponent</th>
                                <th align="center" class="subhdr" scope="col">
                                UCSD Rank</th>
                                <th align="center" class="subhdr" scope="col">
                                Opp Rank</th>
                                <th align="center" class="subhdr" scope="col">
                                Location</th>
                                <th align="center" class="subhdr" nowrap scope=
                                "col">Results</th>
                                <th align="center" class="subhdr" scope="col">
                                Media</th>
                            </tr>
"""
""""""

''

This looks to be the header row of the table. It's the best place to start with scraping and collecting our data, so let's go ahead and do that. As we can see from the above HTML, within the "table" element with class "ScheduleTable", the first "tr" (this stands for "table row") element nested within it is the header. Within that "tr", each of its nested elements (each "th" element) are the header cells. So we need to pull the text body from each of these "th" elements.

This is where BeautifulSoup comes in very hand. BeautifulSoup converts our html into a tree. If element Y is immediately nested within element X, then X is the parent of Y. If Y and Z are both immediately nested within X, then Y and Z are the two children nodes of X in the BeautifulSoup tree.

In [4]:
#let's make beatifulsoup search down until it finds a table with the class "ScheduleTable" and assign this reference to a variable
schedule_table = soup.find("table", {"class": "ScheduleTable"})

#let's print the first couple of lines at this reference point to see what we're working with
for line in list(schedule_table)[:5]:
    print(line)
    
#the formating might look messy, but we're where we want to be



<tr>
<th align="CENTER" class="subhdr" scope="col">
		Date
	</th>
<th align="CENTER" class="subhdr" scope="col">
       		Opponent
    </th>
<th align="CENTER" class="subhdr" scope="col">
		UCSD Rank
	</th>
<th align="CENTER" class="subhdr" scope="col">
		Opp Rank
	</th>
<th align="CENTER" class="subhdr" scope="col">
			Location
		</th>
<th align="CENTER" class="subhdr" nowrap="" scope="col">
			Results
		</th>
<th align="CENTER" class="subhdr" scope="col">
				Media
			</th>
</tr>


<tr>
<td class="odd" colspan="41">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr>
<td align="CENTER" class="subhdr" width="100%">
						Cal State Dominguez Hills Tournament
					</td>
</tr>
</table>
</td>
</tr>




In [5]:
#let's get the header information first
raw_header = []

#we'll tell beautifulsoup to move from our reference point to the first "tr" element using find(), and then we're going to iterate through all the "th" elements in that using findAll()
for x in schedule_table.find("tr").findAll('th'):
    
    #for each "th" element in here, we will extract the text body of it and add it to our raw_header list
    raw_header.append(x.getText())

#if we print it, we should have the header we want (with some messy \n and \t characters thrown in)
print(raw_header)

[u'\n\t\tDate\n\t', u'\n       \t\tOpponent\n    ', u'\n\t\tUCSD Rank\n\t', u'\n\t\tOpp Rank\n\t', u'\n\t\t\tLocation\n\t\t', u'\n\t\t\tResults\n\t\t', u'\n\t\t\t\tMedia\n\t\t\t']


In [6]:
#an easy clean-up trick to remove special characters is to change each string to a list (splitting by spaces), and then recombining it back to a string,
header = [' '.join(x.split()) for x in raw_header]

#our header looks good now! 
print(header)

#don't worry about the "u" in front of each string, that tells us it's unicode, Pandas automatically deals with that

[u'Date', u'Opponent', u'UCSD Rank', u'Opp Rank', u'Location', u'Results', u'Media']


In [7]:
#now it's time to extract the body of the table itself, and add it to our data list
data = []

#we want BeautifulSoup to navigate to iterate through the "tr" (table row) elements, skipping the first (the header)
for row in schedule_table.findAll("tr")[1:]:
    
    #after some look around, we see that rows that do not have data have table elements within them, let's skip those
    if row.find("td").find("table"):
        pass
    
    #otherwise, for this row, we can scrape its text bodies into a list
    else:
        this_row_data = []
        
        #the actual data in each row are in "td" elements, so iterate through those
        for cell in row.findAll("td"):
            
            #let's extract the text, clean it up (as we did at the end of the header) and add to our row list
            this_row_data.append(' '.join(cell.getText().split()))
        
        #and add each row's data to our data holder
        data.append(this_row_data)

#let's see what we have here
for item in data:
    print(item)
        
#it looks pretty, but we still have a couple of rows that do not contain data, let's fix that

[u'Cal State Dominguez Hills Tournament']
[u'Thu, Aug 25', u'Point Loma', u'13', u'-', u'Carson', u'3 - 2 (W) STATS', '']
[u'Thu, Aug 25', u'Bridgeport', u'13', u'-', u'Carson', u'3 - 0 (W) STATS', '']
[u'Fri, Aug 26', u'Concordia (N.Y.)', u'13', u'-', u'Carson', u'3 - 0 (W) STATS', '']
['', '']
[u'Tue, Aug 30', u'Point Loma', u'13', u'-', u'UC San Diego', u'3 - 2 (W) STATS', '']
[u'Cal State San Bernardino Tournament']
[u'Fri, Sep 02', u'St. Martins', u'13', u'-', u'San Bernardino', u'3 - 0 (W) STATS', '']
[u'Fri, Sep 02', u'Hawaii Pacific', u'13', u'8', u'San Bernardino', u'0 - 3 (L) STATS', '']
[u'Sat, Sep 03', u'Northwood', u'13', u'16', u'San Bernardino', u'3 - 0 (W) STATS', '']
[u'Sat, Sep 03', u'Northwest Nazarene', u'13', u'-', u'San Bernardino', u'3 - 0 (W) STATS', '']
['', '']
[u'Thu, Sep 08', u'Grand Canyon', u'10', u'-', u'UC San Diego', u'3 - 1 (W) STATS', '']
[u'Sat, Sep 10', u'Cal State San Bernardino*', u'10', u'2', u'San Bernardino', u'1 - 3 (L) STATS', '']
[u'Fri, Sep

In [8]:
#we can go back and refine our BeautifulSoup search, but it's probably faster to just drop rows without proper data

#first, let's reassign data to only be the rows in data that have length longer than one (thus dropping "Cal State Dominguez Hills Tournament")
data = [x for x in data if len(x) != 1]

#secondly, let's pull out the rows which don't only consist of '' (thus losing the ['', ''] rows)
data = [x for x in data if not all(map(lambda y: y == '', x))]

for row in data:
    print(row)
    
#it looks pretty good now!

[u'Thu, Aug 25', u'Point Loma', u'13', u'-', u'Carson', u'3 - 2 (W) STATS', '']
[u'Thu, Aug 25', u'Bridgeport', u'13', u'-', u'Carson', u'3 - 0 (W) STATS', '']
[u'Fri, Aug 26', u'Concordia (N.Y.)', u'13', u'-', u'Carson', u'3 - 0 (W) STATS', '']
[u'Tue, Aug 30', u'Point Loma', u'13', u'-', u'UC San Diego', u'3 - 2 (W) STATS', '']
[u'Fri, Sep 02', u'St. Martins', u'13', u'-', u'San Bernardino', u'3 - 0 (W) STATS', '']
[u'Fri, Sep 02', u'Hawaii Pacific', u'13', u'8', u'San Bernardino', u'0 - 3 (L) STATS', '']
[u'Sat, Sep 03', u'Northwood', u'13', u'16', u'San Bernardino', u'3 - 0 (W) STATS', '']
[u'Sat, Sep 03', u'Northwest Nazarene', u'13', u'-', u'San Bernardino', u'3 - 0 (W) STATS', '']
[u'Thu, Sep 08', u'Grand Canyon', u'10', u'-', u'UC San Diego', u'3 - 1 (W) STATS', '']
[u'Sat, Sep 10', u'Cal State San Bernardino*', u'10', u'2', u'San Bernardino', u'1 - 3 (L) STATS', '']
[u'Fri, Sep 16', u'Cal State Monterey Bay*', u'12', u'-', u'UC San Diego', u'3 - 0 (W) STATS', '']
[u'Sat, Sep 1

In [9]:
#let's add our data to a Pandas dataframe so we can further clean data and analyze it
df = pd.DataFrame.from_dict(data)

#our dataframe doesn't have column names yet, let's change that by setting them to the header we extract
df.columns = header

#we don't have any values in the Media column, let's get rid of it
df = df.drop(['Media'],axis=1)

#let's get an overview of our dataframe
df

,Date,Opponent,UCSD Rank,Opp Rank,Location,Results
0,"Thu, Aug 25",Point Loma,13,-,Carson,3 - 2 (W) STATS
1,"Thu, Aug 25",Bridgeport,13,-,Carson,3 - 0 (W) STATS
2,"Fri, Aug 26",Concordia (N.Y.),13,-,Carson,3 - 0 (W) STATS
3,"Tue, Aug 30",Point Loma,13,-,UC San Diego,3 - 2 (W) STATS
4,"Fri, Sep 02",St. Martins,13,-,San Bernardino,3 - 0 (W) STATS
5,"Fri, Sep 02",Hawaii Pacific,13,8,San Bernardino,0 - 3 (L) STATS
6,"Sat, Sep 03",Northwood,13,16,San Bernardino,3 - 0 (W) STATS
7,"Sat, Sep 03",Northwest Nazarene,13,-,San Bernardino,3 - 0 (W) STATS
8,"Thu, Sep 08",Grand Canyon,10,-,UC San Diego,3 - 1 (W) STATS
9,"Sat, Sep 10",Cal State San Bernardino*,10,2,San Bernardino,1 - 3 (L) STATS


In [10]:
#let's make a column to show whether the game was won (1) or lost (0), by looking at the (W) or (L) in "Score"
df.loc[ df.Results.str.contains("(W)") , 'Won'] = 1
df.loc[ df.Results.str.contains("(L)") , 'Won'] = 0

#make a column to show which games were conference games, set default to 0 (meaning it's not a conference game)
df['Conference'] = 0

#Rows which have a "*" in 'Opponent" are conference games, let's assign the 'Conference" of those rows to 1
df.loc[ df['Opponent'].str.contains("*",regex=False), 'Conference'] = 1

#let's see the results (of the first 10 rows)
df.head(15)

/Users/dieterjoubert/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/strings.py:207: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)


,Date,Opponent,UCSD Rank,Opp Rank,Location,Results,Won,Conference
0,"Thu, Aug 25",Point Loma,13,-,Carson,3 - 2 (W) STATS,1,0
1,"Thu, Aug 25",Bridgeport,13,-,Carson,3 - 0 (W) STATS,1,0
2,"Fri, Aug 26",Concordia (N.Y.),13,-,Carson,3 - 0 (W) STATS,1,0
3,"Tue, Aug 30",Point Loma,13,-,UC San Diego,3 - 2 (W) STATS,1,0
4,"Fri, Sep 02",St. Martins,13,-,San Bernardino,3 - 0 (W) STATS,1,0
5,"Fri, Sep 02",Hawaii Pacific,13,8,San Bernardino,0 - 3 (L) STATS,0,0
6,"Sat, Sep 03",Northwood,13,16,San Bernardino,3 - 0 (W) STATS,1,0
7,"Sat, Sep 03",Northwest Nazarene,13,-,San Bernardino,3 - 0 (W) STATS,1,0
8,"Thu, Sep 08",Grand Canyon,10,-,UC San Diego,3 - 1 (W) STATS,1,0
9,"Sat, Sep 10",Cal State San Bernardino*,10,2,San Bernardino,1 - 3 (L) STATS,0,1


In [11]:
#let's clean up the Results column by first pulling it out as a list
results = df['Results'].values

print(results)

[u'3 - 2 (W) STATS' u'3 - 0 (W) STATS' u'3 - 0 (W) STATS'
 u'3 - 2 (W) STATS' u'3 - 0 (W) STATS' u'0 - 3 (L) STATS'
 u'3 - 0 (W) STATS' u'3 - 0 (W) STATS' u'3 - 1 (W) STATS'
 u'1 - 3 (L) STATS' u'3 - 0 (W) STATS' u'3 - 0 (W) STATS'
 u'3 - 1 (W) STATS' u'3 - 1 (W) STATS' u'3 - 0 (W) STATS'
 u'1 - 3 (L) STATS' u'1 - 3 (L) STATS' u'3 - 2 (W) STATS'
 u'3 - 2 (W) STATS' u'2 - 3 (L) STATS' u'3 - 0 (W) STATS'
 u'3 - 0 (W) STATS' u'3 - 1 (W) STATS' u'1 - 3 (L) STATS'
 u'2 - 3 (L) STATS' u'0 - 3 (L) STATS' u'3 - 0 (W) STATS']


In [12]:
#our results column is relatively uniform, if we split each string in results by spaces, we just need to get the ints to the left and right of the '-', which are at index 0 and 2
UCSD_results = [int(x.split()[0]) for x in results]
Opp_results = [int(x.split()[2]) for x in results]

print(UCSD_results)
print(Opp_results)

[3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 2, 3, 3, 3, 1, 2, 0, 3]
[2, 0, 0, 2, 0, 3, 0, 0, 1, 3, 0, 0, 1, 1, 0, 3, 3, 2, 2, 3, 0, 0, 1, 3, 3, 3, 0]


In [13]:
#let's add it to our dataframe
df['UCSD Score'] = pd.Series(UCSD_results, index=df.index)
df['Opp Score'] = pd.Series(Opp_results, index=df.index)

#We've extracted the two scores from our original Results, we can drop it now
df = df.drop(['Results'],axis=1)

#let's see what our updated dataframe looks like
df

,Date,Opponent,UCSD Rank,Opp Rank,Location,Won,Conference,UCSD Score,Opp Score
0,"Thu, Aug 25",Point Loma,13,-,Carson,1,0,3,2
1,"Thu, Aug 25",Bridgeport,13,-,Carson,1,0,3,0
2,"Fri, Aug 26",Concordia (N.Y.),13,-,Carson,1,0,3,0
3,"Tue, Aug 30",Point Loma,13,-,UC San Diego,1,0,3,2
4,"Fri, Sep 02",St. Martins,13,-,San Bernardino,1,0,3,0
5,"Fri, Sep 02",Hawaii Pacific,13,8,San Bernardino,0,0,0,3
6,"Sat, Sep 03",Northwood,13,16,San Bernardino,1,0,3,0
7,"Sat, Sep 03",Northwest Nazarene,13,-,San Bernardino,1,0,3,0
8,"Thu, Sep 08",Grand Canyon,10,-,UC San Diego,1,0,3,1
9,"Sat, Sep 10",Cal State San Bernardino*,10,2,San Bernardino,0,1,1,3


In [14]:
#if we want to save our cleaned-up, scraped data, we can write it as a csv
df.to_csv("out.csv",index=False)

In [15]:
"""
We've written a scraper for a single page, but we need to extract info from multiple pages.
Since each page contains all the years of data, so we can scrape all the years available directly from a single page.
And then use those year to get all the pages we want to scrape since their URLs are identical except for the year.
"""

years = []

form_head = soup.find("td", "sm")
form = form_head.find("form")

for opt in form.findAll("option"):
    years.append(opt['value'])
    
#drop the last four chars, the year, to make a general url to add all possibel years to
base_url = url[:-4]

urls_to_scrape = []

for year in years:
    new_url = base_url + year
    urls_to_scrape.append(new_url)
    
for url in urls_to_scrape:
    print(url)

http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2016
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2015
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2014
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2013
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2012
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2011
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2010
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2009
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2008
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=31944&SPID=2334&DB_OEM_ID=5800&Q_SEASON=2007
http://www.ucsdtritons.com/SportSelect.dbml?SPSID=